In [5]:
from albumentations import *      # For data augmentation
import cv2                        # For loading images
import os
from shutil import copyfile
from tqdm import tqdm
import splitfolders

In [4]:
# input_folder = "C:\\Users\\kalin\\Desktop\\Hacatons\\sochi\\Full_dataset_2\\"
# splitfolders.ratio(input_folder, 'C:\\Users\\kalin\\Desktop\\Hacatons\\sochi\\Full_dataset_2', ratio = (0.9, 0.1, 0.0), seed=42, group_prefix=None)

Copying files: 0 files [00:00, ? files/s]

Copying files: 2018 files [00:09, 206.47 files/s]


In [69]:
# transform1 = Compose([
#     HorizontalFlip(always_apply=True),
#     RandomContrast(limit=(-0.4, -0.06), p=0.8),
#     Rotate(limit=(-35, 35), interpolation=4, border_mode=0, p=0.7),
#     MotionBlur(blur_limit=(5, 11), p=0.6)
# ])

In [17]:
transform1 = Compose([
    RandomFog(always_apply=False, p=0.85, fog_coef_lower=0.10, fog_coef_upper=0.10, alpha_coef=0.1),
    Sharpen(always_apply=False, p=0.85, alpha=(0.05, 0.1), lightness=(0.5, 1.0)),
    Spatter(always_apply=False, p=1.0, mean=(0.94, 0.94), std=(50.0, 50.0), gauss_sigma=(2.0, 2.0), intensity=(-0.4, -0.4), cutout_threshold=(0.68, 0.68), mode=['rain']),
    RandomContrast(p=0.85, limit=(-0.40, 0.1))], 
    bbox_params=BboxParams(format='yolo'))

c:\Users\kalin\Desktop\Hacatons\sochi\sochi_hack\.venv\Lib\site-packages\albumentations\augmentations\transforms.py:1284: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [18]:
transform2 = Compose([
    GaussianBlur(p = 0.85, blur_limit=(3,3)),
    ToGray(p = 1.0, ),
    CLAHE(p = 0.85, clip_limit=(1, 4), tile_grid_size=(8,8)),
], bbox_params=BboxParams(format='yolo'))

In [19]:
transform3 = Compose([
    ColorJitter(always_apply=False, p=0.75, brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.2, 0.2)),
    CoarseDropout(always_apply=False, p=0.75, max_holes=100, max_height=4, max_width=4, min_holes=51, min_height=2, min_width=2, fill_value=(0, 0, 0), mask_fill_value=None),
    Downscale(always_apply=False, p=0.75, scale_min=0.9, scale_max=0.9),
], bbox_params=BboxParams(format='yolo'))

c:\Users\kalin\Desktop\Hacatons\sochi\sochi_hack\.venv\Lib\site-packages\albumentations\augmentations\transforms.py:1692: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


In [20]:
transform4 = Compose([
    Emboss(always_apply=False, p=0.75, alpha=(0.2, 0.35), strength=(1.02, 3.0)),
    Equalize(always_apply=False, p=0.75, mode='cv', by_channels=True),
    ISONoise(always_apply=False, p=0.75, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)),
    RGBShift(always_apply=False, p=0.75, r_shift_limit=(-20, 20), g_shift_limit=(-20, 20), b_shift_limit=(-20, 20))
], bbox_params=BboxParams(format='yolo'))

In [21]:
def load_img(path): # Загрузка изображения
    image = cv2.imread(path)
    return image

In [ ]:
common_path = 'C:\\Users\\kalin\\Desktop\\Hacatons\\sochi\\sochi_hack\\BackEnd\ml\\augmentations\\train_org'
path_images = os.path.join(common_path, 'images')  
path_labels = os.path.join(common_path, 'labels')
for filename in tqdm(os.listdir(path_images)):
    image = load_img(os.path.join(path_images, filename))
    with open(os.path.join(path_labels, f'{filename[:-4]}.txt')) as label_file:
        labels = label_file.read()
        labels = labels[1:]
        labels = list(map(float, labels.split()))
        labels.append('0')
        bboxes = []
        bboxes.append(labels)
    transforms = [transform1, transform2, transform3, transform4] # Выбор аугментаций
    for i, el in enumerate(transforms):
        augmented = el(image=image, bboxes=bboxes)
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
    break
        # new_name = f'{filename[:-4]}-{i + 1}'
        # cv2.imwrite(os.path.join(path_images, f'{new_name}.jpg'), augmented_image) # запись нового аугментированного файла
        # copyfile(os.path.join(path_labels, f'{filename[:-4]}.txt'), (os.path.join(path_labels, f'{new_name}.txt'))) #Идет копирование label

In [ ]:
#НЕ ТРОГАЙ!!!

all = './All/'
for index, filename in enumerate(os.listdir(all)):
    image = load_img(os.path.join(all, filename))
    transforms = [transform2]
    for i, el in enumerate(transforms):
        augmented = el(image=image)
        augmented_image = augmented['image']
        cv2.imwrite(os.path.join('./Augmented/', filename), augmented_image)
    if index == 10:
        break